# Experiments on getting prices
* As quickly as possible!

# Findings
* When market is `open`:
    - use `executeAsync` from engine with `price` coro to get prices
    - this is fast
* When market is `closed`:
    - use `ba_async` to get a df of bid-ask-last price
    - this is slow. So data should be trimmed appropriately!


In [1]:
import pathlib
import datetime
import pandas as pd
import numpy as np
import asyncio

from ib_insync import IB, Stock, Contract, util

from collections import defaultdict
from tqdm import tqdm
from datetime import datetime, timedelta

from engine import Vars

In [2]:
util.startLoop()

In [48]:
# * INPUTS
MARKET = 'SNP'
SYMBOL = 'TSLA'
DTE = 10

In [49]:
# * SETTINGS
ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
locals().update(ibp.__dict__)

# set and empty log file
logf = pathlib.Path.cwd().joinpath('data', 'log', 'temp.log')
util.logToFile(path=logf, level=30)
with open(logf, "w"):
    pass

datapath = pathlib.Path.cwd().joinpath('data', MARKET.lower())

In [50]:
df_opts = pd.read_pickle(datapath.joinpath('df_opts.pkl'))
df_unds = pd.read_pickle(datapath.joinpath('df_unds.pkl'))
df_chains = pd.read_pickle(datapath.joinpath('df_chains.pkl'))

  0%|                                                                                          | 0/2 [1:04:06<?, ?it/s]


## Preparing contracts
### Nearest DTE contracts - sorted by strikeDelta

In [51]:
df_sym = df_opts[df_opts.symbol == SYMBOL] # filter symbol
df_dte = df_sym[df_sym.dte == df_sym.dte.unique().min()] # filter dte

# sort from strike nearest to undPrice to farthest
df_dte = df_dte.iloc[abs(df_dte.strike-df_dte.undPrice.iloc[0]).argsort()]
contracts = df_dte.contract.unique()

contracts = contracts[:200] # !!! DATA LIMITER

print(f"\nLength of df_dte is: {len(df_dte)}")

df_dte.head()


Length of df_dte is: 218


,symbol,dte,strike,expiry,secType,conId,right,contract,lot,und_iv,undPrice,fall,rise,rsi
107322,TSLA,5,695.0,20210108,OPT,461170870,C,"Option(conId=461170870, symbol='TSLA', lastTra...",100,0.658888,694.78,172.61,120.43,68.952786
107321,TSLA,5,695.0,20210108,OPT,461170891,P,"Option(conId=461170891, symbol='TSLA', lastTra...",100,0.658888,694.78,172.61,120.43,68.952786
107319,TSLA,5,690.0,20210108,OPT,458060071,P,"Option(conId=458060071, symbol='TSLA', lastTra...",100,0.658888,694.78,172.61,120.43,68.952786
107320,TSLA,5,690.0,20210108,OPT,458059907,C,"Option(conId=458059907, symbol='TSLA', lastTra...",100,0.658888,694.78,172.61,120.43,68.952786
107324,TSLA,5,700.0,20210108,OPT,458059913,C,"Option(conId=458059913, symbol='TSLA', lastTra...",100,0.658888,694.78,172.61,120.43,68.952786


### Option strike closest to underlying

In [52]:
df1 = df_dte[:1]
contract = df1.contract.iloc[0]

df1

,symbol,dte,strike,expiry,secType,conId,right,contract,lot,und_iv,undPrice,fall,rise,rsi
107322,TSLA,5,695.0,20210108,OPT,461170870,C,"Option(conId=461170870, symbol='TSLA', lastTra...",100,0.658888,694.78,172.61,120.43,68.952786


# Getting prices

## A) Mix of historical and real-time prices

- Using a mix of `reqHistoricalTicks` and `reqMktData`

### a) Multiple contracts using custom async def

In [88]:
from collections import namedtuple
from engine import price
from support import get_prec

async def qpCoro(ib: IB, contract: Contract, **kwargs) -> pd.DataFrame:
    """Coroutine for quick price from market | history"""
    
    try:
        FILL_DELAY = kwargs["FILL_DELAY"]
    except KeyError as ke:
        print(
            f"\nWarning: No FILL_DELAY supplied! 5.5 second default is taken\n"
        )
        FILL_DELAY = 5.5
    
    if isinstance(contract, tuple):
        contract = contract[0]
    
    df_mktpr = await price(ib, contract, **kwargs)
    
    async def histCoro():
        
        result = defaultdict(dict)
        
        try:
            ticks = await asyncio.wait_for(ib.reqHistoricalTicksAsync(
                            contract=contract,
                            startDateTime="",
                            endDateTime=datetime.now(),
                            numberOfTicks=1,
                            whatToShow="Bid_Ask",
                            useRth=False,
                            ignoreSize=False), timeout=None)

        except asyncio.TimeoutError:
            tick = namedtuple('tick', ['time', 'priceBid', 'priceAsk'])
            ticks = [tick(time=pd.NaT, priceBid=np.nan, priceAsk=np.nan)]

        # extract bid and ask price, if available!
        try:
            bid_ask = ticks[-1]  # bid ask is not availble for Index securities!
            result["bid"] = bid_ask.priceBid
            result["ask"] = bid_ask.priceAsk
            result["batime"] = bid_ask.time

        except IndexError:
            result["bid"] = np.nan
            result["ask"] = np.nan
            result["batime"] = pd.NaT
        
        return result
    
    # bid/ask with -1.0 as market is not open
    if (df_mktpr.bid.iloc[0] == -1.0) or (df_mktpr.ask.iloc[0] == -1.0):
        result = await histCoro()
        
        df_pr = df_mktpr.assign(batime = result['batime'],
                    bid = result['bid'],
                    ask = result['ask'])
    else:
        df_mktpr['batime'] = df_mktpr['time']
        df_pr = df_mktpr
        
    # use bid-ask avg if last price is not available
    df_pr = df_pr.assign(price=df_pr["last"]\
                 .combine_first(df_pr[["bid", "ask"]]\
                 .mean(axis=1)))
    
    df_pr = df_pr.sort_values(['right', 'strike'], ascending=[True, False])
    
    return df_pr


In [89]:
async def qpAsync(ib:IB, contracts, **kwargs) -> pd.DataFrame:
    """Quick Price with bid-ask for a number of contracts"""
    
    if hasattr(contracts, '__iter__'):
        tasks = [qpCoro(ib=ib, contract=contract, **kwargs) for contract in contracts]
    else:
        tasks = [qpCoro(ib=ib, contract=contracts, **kwargs)]
        
    df_prs = [await res for res in tqdm(asyncio.as_completed(tasks), total=len(tasks))]
    df = pd.concat(df_prs, ignore_index=True)
    return df

In [90]:
with IB().connect(HOST, PORT, CID) as ib:
    df_pr = ib.run(qpAsync(ib, contracts, **{'FILL_DELAY': 5.5}))
    ib.disconnect()





  0%|                                                                                            | 0/5 [00:00<?, ?it/s]



100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:06<00:00,  1.30s/it]


### b) Single contract

In [91]:
%%time
with IB().connect(HOST, PORT, CID) as ib:
    df_pr = ib.run(qpAsync(ib=ib, contracts = contract, **{'FILL_DELAY': 5.5}))





  0%|                                                                                            | 0/1 [00:00<?, ?it/s]



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.26s/it]

Wall time: 6.33 s


In [92]:
df_pr

,secType,conId,symbol,expiry,strike,right,localSymbol,contract,time,greeks,bid,ask,close,last,price,iv,batime
0,OPT,461170870,TSLA,20210108,695.0,C,TSLA 210108C00695000,"Option(conId=461170870, symbol='TSLA', lastTra...",2021-01-05 07:43:27.374591+00:00,None,42.05,42.85,42.55,NaN,42.45,NaN,2021-01-04 20:59:59+00:00


## B) Only From Market data
### 1) Engine price
#### a) Single contract - with 8 second fill delay

In [ ]:
%%time
from engine import price

with IB().connect(HOST, PORT, CID) as ib:
    df = ib.run(price(ib, contract, **{'FILL_DELAY': 8}))

df

#### b) Multiple contracts - with 8 second fill delay
* Takes about 10 seconds for 200 prices
* **`NOTE:`**
    - should be used only when the market is open
    - only gives `last` price, when the market is closed!
    - this `last` price is not accurate.

In [ ]:
%%time
from engine import price, pre_process, make_name, executeAsync, post_df

with IB().connect(HOST, PAPER, CID) as ib:
    
    ib.client.setConnectOptions('+PACEAPI')
    
    df = ib.run(
            executeAsync(
                ib=ib,
                algo=price,
                cts = contracts,
                CONCURRENT = 200,
                TIMEOUT=8,
                post_process=post_df,
                SHOW_TQDM=True,
                **{'FILL_DELAY': 8}
            ))

In [ ]:
# sort by closest to strike
df = df.assign(undPrice=df_unds[df_unds.symbol == SYMBOL].undPrice.iloc[0])
df = df.iloc[abs(df.strike-df.undPrice.iloc[0]).argsort()]

# remove options without time
df1 = df[~df.time.isnull()]
df1

## C) from engines' `OHLCs`
### 1) Using `OHLC` with executeAsync for `bid_ask` price
`NOTE`
* Doesn't work well. Gets stuck on some options contract without hist bars!

#### a) Single contract

#### b) Multiple contracts
1. Using existing `ohlc` function is still slow and very erratic. It is better to avoid this.
2. Data gets duplicated. Hence drop_duplicates and filtering of nans are needed

## D) Bid-Ask - with `reqHistoricalData` for multiple contracts

**`NOTE`**
* This function is very slow, but gives bid-ask-last price when market is closed
* Takes 2 minutes for 20 prices!

In [ ]:
df_dte

### 3) Ask `trade` prices from OHLC

In [ ]:
async def hist_async(ib, contracts):
        
    # request data from start date
    async def coro(c):

        start = (datetime.utcnow()- timedelta(days=1)).date()
        end = datetime.utcnow().date()

        barsList = []
        dt = end

        while dt > start:

            bars = await ib.reqHistoricalDataAsync(contract,
                                            endDateTime=dt,
                                            durationStr='1 D',
                                            barSizeSetting='1 day',
                                            whatToShow='TRADES',
                                            useRTH=True,
                                            formatDate=2)
            if not bars:
                break
            barsList.append(bars)
            dt = bars[0].date

        return bars

    hist = await asyncio.gather(*[coro(c) for c in contracts])

    return hist